In [2]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
import time
import bs4
import re
import os 
import urllib.request, urllib.error, urllib.parse

In [281]:
# Read the dataset with the profile links
profile_urls = pd.read_csv('RSC reviews with profile ids.csv')

In [282]:
profile_urls.head()

,source,product,PART NUMBER_custom,SKU_custom,analysis_purpose_custom_custom,flag_custom,special_name_custom,test_field2_custom,test_field3_custom,name,...,statusdt,statustime,helpfulcount,commenttext,commentauthor,officialcomment,totalcomments,commentts,commentdatestring,inputtime
0,amazon.ca,B078N8NR7G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PetSafe 900 Meter Remote Trainer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/22/18 6:24
1,amazon.ca,B01HO8U5NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drinkwell Platinum Pet Fountain 168oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/15/19 10:32
2,amazon.ca,B01HO8U5NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drinkwell Platinum Pet Fountain 168oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/4/19 12:02
3,amazon.ca,B01HO8U5NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drinkwell Platinum Pet Fountain 168oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/2/19 20:35
4,amazon.ca,B01HO8U5NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drinkwell Platinum Pet Fountain 168oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/31/18 8:00


In [283]:
profile_urls.columns

Index(['source', 'product', 'PART NUMBER_custom', 'SKU_custom',
       'analysis_purpose_custom_custom', 'flag_custom', 'special_name_custom',
       'test_field2_custom', 'test_field3_custom', 'name', 'date', 'status',
       'sentiment', 'topic', 'notes', 'profile', 'Unnamed: 16', 'author',
       'verified', 'vine', 'stars', 'pvotes', 'tvotes', 'title', 'text',
       'image', 'video', 'reviewid', 'reviewlink', 'parent', 'statusdt',
       'statustime', 'helpfulcount', 'commenttext', 'commentauthor',
       'officialcomment', 'totalcomments', 'commentts', 'commentdatestring',
       'inputtime'],
      dtype='object')

In [284]:
# use for later aggregation
profile_urls = profile_urls[['author','source','product','profile']]

In [285]:
profile_urls['source_product'] = profile_urls['source'] +' '+ profile_urls['product']

In [286]:
profile_urls.head()

,author,source,product,profile,source_product
0,Amazon Customer,amazon.ca,B078N8NR7G,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B078N8NR7G
1,nathalie,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC
2,Conure Mum,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC
3,Wayne Smith,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC
4,Rob Self,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC


# divide the web-scraping tasks

In [287]:
profile_urls_sub1 = profile_urls.dropna()

In [288]:
profile_urls_sub1 = profile_urls_sub1[profile_urls_sub1['profile'].str.contains('account')]

In [289]:
len(profile_urls_sub1)

26769

In [302]:
# First 750 products -- 14240 rows
profile_urls_Iris = profile_urls_sub1.head(13382)

In [304]:
profile_urls_Iris.head()

,author,source,product,profile,source_product
0,Amazon Customer,amazon.ca,B078N8NR7G,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B078N8NR7G
1,nathalie,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC
2,Conure Mum,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC
3,Wayne Smith,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC
4,Rob Self,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC


In [297]:
profile_urls_Khaled = profile_urls_sub1.tail(len(profile_urls_sub1)-len(profile_urls_Iris))

In [298]:
profile_urls_Khaled

,author,source,product,profile,source_product
26513,debzlouisep,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26514,Martin Britnell,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26515,Cat Woman,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26516,Bazza,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26517,June Rowlands,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26518,Amazon Customer,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26519,Dave,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26520,Linda Colwell,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26521,BrownDog,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI
26522,JTW,amazon.uk,B00B17ETPI,https://www.amazon.co.uk/gp/profile/amzn1.acco...,amazon.uk B00B17ETPI


# generating the list 

In [239]:
source_product = profile_urls_sub1[(profile_urls_sub1['source'] != 'amazon.de') & (profile_urls_sub1['source'] != 'amazon.fr')]

In [79]:
source_product = pd.DataFrame(source_product.groupby(['source','product']).count())

In [80]:
source_product = source_product.reset_index()

In [81]:
source_product = source_product.sort_values(['source','profile'],ascending=False).reset_index(drop = True)

In [83]:
source_product.to_csv('source_product.csv')

In [306]:
profile_urls1 = profile_urls.head(20)

# web-scraping

In [314]:
for i in range(0,11):
    time.sleep(1) #Hold 1 seconds before the next scrape.
    num=str(i)
    newurl = profile_urls1.loc[i,'profile']
    d = webdriver.Chrome(executable_path=os.path.abspath('chromedriver')) 
    d.get(newurl)
    d.execute_script('document.documentElement.scollTop=10000')# To make the webdriver scroll down to fetch as many as the reviews.
    time.sleep(10) # sleep again the let the page load
    name='profile'+num+'.txt' #The new file name. 
    file=open(name,'w') #Create a new file.
    file.write(d.page_source) #Write the html requested into it.
    file.close()
    d.close()#Close the google webpage that webdriver open for you, otherwise it will be crazy.

# extract attributes

In [316]:
soup = {}
for i in range(0,11):
    num=str(i)
    name='profile'+num+'.txt'
    text=open(name,'r').read() #Read the text from the file.
    soup[i]=bs4.BeautifulSoup(text) #Create a beautifulsoup object using the txt we got.

In [317]:
# generating new columns: one reviewer one row
for i in range(0,11):
#for i in profile_urls.index.unique():
    tag0 = soup[i].find_all('div', class_='dashboard-desktop-stat-value')[0] 
    profile_urls.loc[i,'helpful_votes'] = tag0.find('span', class_='a-size-large a-color-base').get_text() 
    
    for tag in soup[i].find_all('div', class_='a-row a-spacing-none name-container'):    
        profile_urls.loc[i,'name'] = tag.find('span', class_='a-size-extra-large').get_text() 
    
    for tag in soup[i].find_all('div', class_='dashboard-desktop-stat-value'):    
        profile_urls.loc[i,'num_of_reviews'] = int(tag.find('span', class_='a-size-large a-color-base').get_text())
    
    profile_urls.loc[i,'num_of_reviews_count'] = len(soup[i].find_all('div', class_='a-section profile-at-content'))
    # take back
    profile_urls['take_back'] = profile_urls.apply(lambda x: x['num_of_reviews'] - x['num_of_reviews_count'], axis=1)
    if (profile_urls.loc[i,'num_of_reviews_count'] < 120) & (profile_urls.loc[i,'take_back'] > 0):
        profile_urls.loc[i,'take_back_01'] = 1
    else:
        profile_urls.loc[i,'take_back_01'] = 0
    
    
    verified = []
    for tag in soup[i].find_all('div', class_='a-row a-spacing-mini'): 
        try:   
            verified.append(tag.find('span', class_='a-size-small a-color-state profile-at-review-badge a-text-bold').get_text())
        except:
            continue  
        profile_urls.loc[i,'num_of_verified'] = len(verified)
    
    profile_urls.loc[i,'num_of_unverified'] = profile_urls.loc[i,'num_of_reviews_count'] - profile_urls.loc[i,'num_of_verified']
    
    date_mode_number = []
    # mode_number means if one person has many purchases on one day, how many purchases? I found out the date that appears most times.
    for tag in soup[i].find_all('div', class_='a-profile-content'):
        date_mode_number.append(tag.find('span', class_='a-profile-descriptor').get_text())
        profile_urls.loc[i,'mode_number'] = len([j for j, review in enumerate(date_mode_number) if review == max(set(date_mode_number), key=date_mode_number.count)])
    if profile_urls.loc[i,'mode_number'] > 20:
        profile_urls.loc[i,'samedate_20'] = 1
    else:
        profile_urls.loc[i,'samedate_20'] = 0
    
    # reviewer anonymous
    if ('Customer' in profile_urls.loc[i,'author']) | ('customer' in profile_urls.loc[i,'author']):
        profile_urls.loc[i,'anonymous'] = 1
    else:
        profile_urls.loc[i,'anonymous'] = 0
    # only 5 star reviews
    star5 = []
    for tag in soup[i].find_all('div',class_='a-section a-spacing-mini'):
        star5.append(tag.find('span',class_='a-icon-alt').text)
    if (len(set(star5)) == 1) & ('5 out of five stars' in set(star5)):
        profile_urls.loc[i,'only_5star'] = 1
    else:
        profile_urls.loc[i,'only_5star'] = 0
    
    
    

In [318]:
profile_urls

,author,source,product,profile,source_product,helpful_votes,name,num_of_reviews,num_of_reviews_count,take_back,take_back_01,num_of_verified,num_of_unverified,mode_number,samedate_20,anonymous,only_5star
0,Amazon Customer,amazon.ca,B078N8NR7G,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B078N8NR7G,1,Amazon Customer,4.0,4.0,0.0,0.0,4.0,0.0,3.0,0.0,1.0,0.0
1,nathalie,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,0,nathalie,18.0,10.0,8.0,1.0,10.0,0.0,5.0,0.0,0.0,1.0
2,Conure Mum,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,50,Conure Mum,161.0,10.0,151.0,1.0,8.0,2.0,2.0,0.0,0.0,0.0
3,Wayne Smith,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,0,Wayne Smith,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
4,Rob Self,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,3,Rob Self,7.0,7.0,0.0,0.0,7.0,0.0,5.0,0.0,0.0,0.0
5,Richard Goods,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,5,Richard Goods,51.0,51.0,0.0,0.0,50.0,1.0,27.0,1.0,0.0,0.0
6,Jerome Tanguay,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,0,Jerome Tanguay,21.0,10.0,11.0,1.0,10.0,0.0,5.0,0.0,0.0,0.0
7,Tammy Roode,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,0,Tammy Roode,4.0,4.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,1.0
8,Lisa,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,25,Lisa,98.0,70.0,28.0,1.0,67.0,3.0,12.0,0.0,0.0,0.0
9,mika jiang,amazon.ca,B01HO8U5NC,https://www.amazon.ca/gp/profile/amzn1.account...,amazon.ca B01HO8U5NC,0,mika jiang,5.0,5.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0


# aggregate again for products
one product one row

In [319]:
product_df = pd.DataFrame(profile_urls['product','source'].unique(),columns = ['product'])

KeyError: ('product', 'source')

In [ ]:
product_df 

## still need adjustment (aggregate on ['source','product'])

In [276]:
# when the reviewer have over 20 reviews on the same date
for i in product_df.index.unique():
    product_df.loc[i, 'samedate_20'] = len(profile_urls[profile_urls['product'] == product_df.loc[i,'product']][profile_urls['source'] == product_df.loc[i,'source']][profile_urls['samedate_20'] == 1])/len(profile_urls[profile_urls['product'] == product_df.loc[i,'product']])
    
# helpful votes (sum of helpful votes / sum of number of reviews)
profile_urls['helpful_votes'] = profile_urls['helpful_votes'].astype(int)
for i in product_df.index.unique():
    product_df.loc[i, 'helpful_votes'] = round((profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['helpful_votes'].sum())/profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['num_of_reviews'].sum(),2)
    
# one-hit wonderpercentage (percentage of reviewers who only has one review history)
for i in product_df.index.unique():
    product_df.loc[i, 'one_hit'] = round((profile_urls[(profile_urls['product'] == product_df.loc[i,'product']) & (profile_urls['num_of_reviews'] == 1)]['product'].count()) / (profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['product'].count()),2)
            
# take-back reviewer (here, in terms of speed, we can only look at take-back reviewers with less than 120 reviews)
for i in product_df.index.unique():
    product_df.loc[i, 'take_back'] = round((profile_urls[(profile_urls['product'] == product_df.loc[i,'product']) & (profile_urls['take_back_01'] == 1)]['product'].count()) / (profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['product'].count()),2)

# percentage of verified purchases (here we use num_of_reviews_count as the denominator because these two columns are what we count, not the number on the top of the page)
for i in product_df.index.unique():
    product_df.loc[i, 'verified_purchase'] = round((profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['num_of_verified'].sum())/profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['num_of_reviews_count'].sum(),2)

# reviewer anonymous
for i in product_df.index.unique():
    product_df.loc[i, 'reviewer_anonymous'] = round((profile_urls[(profile_urls['product'] == product_df.loc[i,'product']) & (profile_urls['anonymous'] == 1)]['product'].count()) / (profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['product'].count()),2)

# only 5 star reviews
for i in product_df.index.unique():
    product_df.loc[i, 'only_5star'] = round((profile_urls[(profile_urls['product'] == product_df.loc[i,'product']) & (profile_urls['only_5star'] == 1)]['product'].count()) / (profile_urls[profile_urls['product'] == product_df.loc[i,'product']]['product'].count()),2)


            
        
    
    
    

KeyError: 'source'

In [234]:
product_df

,product,samedate_20,helpful_votes,one_hit,take_back,verified_purchase,reviewer_anonymous,only_5star
0,B078N8NR7G,0.0,0.25,0.0,0.0,1.00,1.0,0.0
1,B01HO8U5NC,0.1,0.18,0.0,0.1,0.96,0.0,0.2
